In [60]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
import math
import time
import datetime
import os

In [45]:
#旅遊部落客排行榜
blog_name = []
rank = 0
for i in range(7):
    page = i + 1
    r = requests.get('https://www.pixnet.net/blog/bloggers/category/16/{}'.format(page))
    r.encoding = 'utf8'
    soup = bs(r.text, 'lxml')
    for j in range(len(soup.select('#content .author'))):
        blog_name.append(soup.select('.author')[j]['href'].split('/')[-1])
        rank += 1
        if rank >= 100:
            break
blog_name

['lohas',
 'wkitty',
 'bajenny',
 'judyer101010',
 'smilejean',
 'mimg47',
 'niniblue',
 'drugs',
 'paine0602',
 'ch3126',
 'jay7134',
 'cheng0073',
 'bangkokgoplay',
 'john547',
 'barbieedoll',
 'oldfather1029',
 'viviyu',
 'crystal1983',
 'fbuon2881',
 'venuslin0113',
 'windko0813',
 'j272popo',
 'agathachiu66',
 'l50740',
 'pingpig',
 'sweetmemory99',
 'maymay1231',
 'kakakuksk',
 'banbi217',
 'nana0032458',
 'kavanachang',
 'enlife',
 'cat875030',
 'jay0909',
 'difeny',
 'a59407908',
 'jimmraz',
 'evshhips',
 'coupine650520',
 'demi0130',
 'masaharuwu',
 'rita11836',
 'as660707',
 'happyday760626',
 'tainanlohas',
 'julia116',
 'yopalit',
 'amarylliss',
 'monkey2010',
 'dreamycat0421',
 'vilily',
 'gugugu4411',
 'damon624',
 'mika0310',
 'dong1104',
 'catty0914',
 'hrylin',
 'nanacobaby',
 'aliciaisme',
 'qpjj',
 'bonddealerbook',
 'dailin114',
 'darrenje0322',
 'may0708',
 'candybear0121',
 'aura10',
 'sinea100',
 'nini710',
 'bulanini',
 'louis5149',
 'ballenf',
 'eeooa0314',
 't

In [82]:
payload = {
    "user":"",
    "page":""
}
filename = "Pixnet-"+datetime.datetime.now().strftime('%Y%m%d')+".csv"

for bn in blog_name:
    print (bn)
    articles = []
    payload['user'] = bn
    payload['page'] = "1"
    re = requests.get('https://emma.pixnet.cc/blog/articles', params = payload)
    totalpages = math.ceil(pd.read_json(re.text)['total'][0]/pd.read_json(re.text)['per_page'][0])

    for i in range(totalpages):
        payload['page'] = i + 1
        re = requests.get('https://emma.pixnet.cc/blog/articles', params = payload)
        pd1 = pd.read_json(re.text)['articles']
        pd2 = pd.DataFrame.from_records(pd1)
        pd2['hits_total'] = 0
        pd2['pub_date'] = 'na'
        pd2['blog_name'] = payload['user']
        pd2['body'] = 'na'

        for j in range(len(pd1)):
            pd2['hits_total'].iloc[j] = pd2['hits'][j]['total']
        for k in range(len(pd2)):
            pd2['pub_date'].iloc[k] = datetime.date.fromtimestamp(int(pd2['public_at'][k])).strftime('%Y-%m-%d')
        #取得文章內容
        for l in range(len(pd1)):
            user = payload['user']
            r = requests.get('https://emma.pixnet.cc/blog/articles/{}'.format(pd2['id'][l]), params = {'user':user} )
            pd_body = pd.read_json(r.text)['article']['body']
            soup_body = bs(pd_body, 'lxml')
            pd2['body'].iloc[l] = soup_body.text.replace("\n", "")

        articles.append(pd2[['blog_name', 'category', 'category_id', 'site_category', 'site_category_id', \
                             'id', 'link', 'title', 'pub_date', 'hits_total', 'body']])
        Pixnet_articles = pd.concat(articles, ignore_index=True)
        if not os.path.isfile(filename):
            Pixnet_articles.to_csv(filename)
        else:
            Pixnet_articles.to_csv(filename, mode='a', header=False)
        Pixnet_articles.clear()

lohas


C:\Miniconda2\envs\hour_of_code\lib\site-packages\pandas\core\indexing.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '遠端主機已強制關閉一個現存的連線。', None, 10054, None))

In [43]:
#近期熱門部落格
r = requests.get('https://emma.pixnet.cc/mainpage/blog/categories/hot_weekly/29?per_page=100')
hot_user_name = pd.read_json(r.text)['user_name']
hot_user_name

0        benshee1005
1            whenwas
2         rosecocoya
3          cline1413
4      purpleswallow
5        Crystal1983
6          tammytwkr
7       judyer101010
8          meiko1101
9       naughtyangel
10        uniquevera
11            ch3126
12            wkitty
13       annybear920
14             lohas
15            ectomy
16              Sybs
17        mini692565
18      venuslin0113
19         chahabi77
20        eddie80128
21          zoey0830
22           bajenny
23        TWNLCC2012
24         antiwhite
25        kaikai0603
26    missrachelnina
27         kakakuksk
28      angeline5775
29             drugs
           ...      
70          twaychan
71           peko721
72        yukiko1437
73          miihuang
74         dailin114
75        fafa710117
76       eggface0507
77             anise
78      Estrella8850
79         stellahyc
80            a1630a
81        inlifefood
82        Whitesuper
83         rita11836
84            Dorcas
85          arielctl
86           

In [81]:
payload = {
    "user":"lohas",
    "page":"1",
    "per_page":"3"
}
for i in range(2):
    payload['page'] = i + 1
    articles = []
    re = requests.get('https://emma.pixnet.cc/blog/articles', params = payload)
    pd1 = pd.read_json(re.text)['articles']
    pd2 = pd.DataFrame.from_records(pd1)
    pd2['hits_total'] = 0
    pd2['pub_date'] = 'na'
    pd2['blog_name'] = payload['user']
    pd2['body'] = 'na'
    for j in range(len(pd1)):
        pd2['hits_total'].iloc[j] = pd2.loc[j,'hits']['total']
    for k in range(len(pd1)):
        pd2['pub_date'].iloc[k] = datetime.date.fromtimestamp(int(pd2['public_at'][k])).strftime('%Y-%m-%d')

    #取得文章內容
    for l in range(len(pd1)):
        user = payload['user']
        r = requests.get('https://emma.pixnet.cc/blog/articles/{}'.format(pd2['id'][l]), params = {'user':user} )
        pd_body = pd.read_json(r.text)['article']['body']
        soup_body = bs(pd_body, 'lxml')
        pd2['body'].iloc[l] = soup_body.text.replace("\n", "")
        
    articles.append(pd2[['blog_name', 'category', 'category_id', 'site_category', 'site_category_id', \
                                                    'id', 'link', 'title', 'pub_date', 'hits_total', 'body']])
    Pixnet_articles = pd.concat(articles, ignore_index=True)
    if not os.path.isfile("test.csv"):
        Pixnet_articles.to_csv("test.csv")
    else:
        Pixnet_articles.to_csv("test.csv", mode='a', header=False)

C:\Miniconda2\envs\hour_of_code\lib\site-packages\pandas\core\indexing.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [42]:
r = requests.get('https://emma.pixnet.cc/blog/articles/{}'.format('43446004'), params = {'user':'lohas'} )
pd_body = pd.read_json(r.text)['article']['body']
soup_body = bs(pd_body, 'lxml')
article_body = soup_body.text.replace("\n", "")
article_body

'手塚治虫の 鉄腕アトム 原子小金鋼，可是大方小時候最喜歡的卡通人物之一，關西經濟連合會將在 2016年04月08日發售全新的 原子小金鋼版 KANSAI ONE PASS 関西ワンパス ，KANSAI ONE PASS 本身就是一張 ICOCA ，當然也能在日本全國使用，在關西地區不少的觀光設施，還有優惠價格唷，讓你來日本關西玩，還能享有優惠唷。原子小金鋼版 KANSAI ONE PASS 関西ワンパス 將在 2016年04月08日發售，是針對來日本旅遊，持短期簽證外國人販售的，日本人並不能買。卡片的金額為 3000 日幣，包含了卡的押金500日幣 + 可用金額 2500日幣，在關西機場、JR西日本、阪急電鐵、近畿日本鐵道、大阪地鐵等關西的９條交通路線，外國人持護照均可以買到這款KANSAI ONE PASS 関西ワンパス。 (以現場實際銷售為主)銷售地點如下：http://kansaionepass.com/kf_pr/kf_pr_sale_tw.html\xa0(表格資料來源)區鐵路名銷售地點關西國際機場JR西日本關西機場站南海電鐵關西機場站票務處京\u3000都JR西日本京都站阪急電鐵阪急京都 旅遊咨詢服務中心（河原町）京阪電氣鐵道三條站近畿日本鐵道京都站京都市交通局城市公交和地鐵信息（京都站前）城市公交和地鐵信息（Kotochika京都）大\u3000阪JR西日本新大阪站阪急電鐵阪急旅客服務中心 大阪・梅田京阪電氣鐵道京橋站近畿日本鐵道大阪難波站大阪上本町站大阪阿部野橋站南海電鐵難波站2層服務中心大阪市交通局新大阪站中檢票口“Subway Information Counter”御堂筋線難波站南北檢票口「Subway Information Counter」神\u3000戶阪神電気鉄道阪神電車服務中心（神戶三宮）神戶市交通局神戶市營巴士・地鐵\u3000客戶服務處（三宮）中文官網介紹：http://kansaionepass.com/kf_pr/kf_pr_tw.html目前卡片是採限量實驗性銷售，販售日期為2016 年 4 月 8 日（金）～2017 年 3 月 31 日（金），新聞資料來源：http://www.nkiac.co.jp/news/2015/2324/kansaionepass.pdf原子小金鋼版 KANSAI ONE PASS 関西ワン